# CNN for Cifar-10 Dataset

## Introduction

In this first notebook we are going to explain how we have set up our work.  
Essentially the process is divided into several parts:
* Set up environment with the Cifar-10 Dataset
* Define a convolutional neural network
* Define a quantization method
* Train the convolutional neural network
* Provide information about CNN's performance and accuracy

### Requirements

For this project we are used:
* Python 3.6
* Tensorflow 1.8.0
* *GPUs parallel calculation manager* nVidia CUDA 9.0
* *GPU-accelerated library* nVidia cuDNN 7.1
* or CPU optimized tensorflow for intel

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import graph_io, graph_util
from tensorflow.tools.graph_transforms import TransformGraph

from cnn.dense import (NET_NAME, dataset_preprocessing_by_keras, eval_fn,
                       forward_pass, loss_fn)
from cnn.model_class import MODELS_DIR, TfClassifier
from cnn.utils.dataset import load_cifar10
from cnn.utils.save_models import load_frozen_graph

/home/daibak/.virtualenvs/aca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Cifar-10 Dataset

Cifar-10 Dataset is taken from the official website www.cs.toronto.edu.

Dataset is stored in the data directory: cnn/data. From Cifar-10 dataset we are going to take x_train, t_train, x_test and t_test.
The training dataset set is used for training the CNN, the testing dataset is used for evaluate the performance and the accuracy of the network.

In [2]:
x_train, t_train, x_test, t_test = load_cifar10()

In [3]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

It also needs to be normalized

In [4]:
x_train = dataset_preprocessing_by_keras(x_train)
x_train[0, :, :, 0]

array([[-0.68747891, -0.65572952, -0.63985482, ..., -0.95734874,
        -1.00497283, -0.92559935],
       [-0.65572952, -0.63985482, -0.59223073, ..., -0.92559935,
        -1.02084753, -0.86210057],
       [-0.7192283 , -0.63985482, -0.65572952, ..., -0.68747891,
        -0.735103  , -0.70335361],
       ...,
       [-0.36998499, -0.7509777 , -1.00497283, ..., -0.25886212,
        -0.24298742, -0.14773924],
       [-0.25886212, -0.65572952, -1.19546919, ...,  0.31262694,
        -0.00486698, -0.49698256],
       [-0.27473681, -0.35411029, -1.11609571, ...,  0.75711843,
         0.64599556,  0.28087755]])

## CNN Model and Training

We will use a custom made wrapper for tensorfllow NN training and use

This CNN is called *dense_cnn*. Here we will explain how it is composed.

The CNN is composed by several layers. In the first part there are 2 **convolutional** layers and 2 **pooling** layers (they are alternated), then there are a *flatten* layer followed by a **relu** layer, a *dropout* layer and finally a **softmax** layer.

The network uses a stochastic gradient descent optimizer and a categorical crossentropy loss.  
To judge the performance of our model we are used a MSE metric.

In [2]:
model = TfClassifier(NET_NAME, forward_pass, loss_fn, eval_fn,
                     tf.train.AdamOptimizer())

This network is trained for 50 epochs.

In [9]:
history = model.fit(
    [x_train, t_train],
    batch_size=64,
    validation_split=0.1,
    epochs=1,
    verbosity=1)

print(history)

For training: tensorboard --logdir=/tmp/log-tb/dense_cnn/training
For validation: tensorboard --logdir=/tmp/log-tb/dense_cnn/validation
{'logits': (array([[-1.2727997 , -4.118344  ,  3.0404963 , ...,  0.434211  ,
        -5.4986506 , -4.9266114 ],
       [ 0.67287916,  5.107691  , -1.9640099 , ..., -2.370006  ,
         0.5111373 ,  1.3213307 ],
       [-0.58712345, -4.8890724 ,  2.3193388 , ...,  0.4489386 ,
        -2.6303477 , -4.485857  ],
       ...,
       [-2.5256755 , -2.4960575 ,  0.04580361, ...,  1.2207695 ,
        -2.8485522 , -1.5472312 ],
       [ 0.07853588, -2.267046  , -1.6271029 , ...,  1.9091396 ,
        -2.5177221 , -2.9749494 ],
       [ 1.2112377 , -0.310529  , -0.02666046, ..., -2.1066892 ,
         5.757359  , -1.0108342 ]], dtype=float32),), 'classes': (array([4, 1, 4, ..., 3, 4, 8]),), 'probabilities': (array([[3.6865592e-03, 2.1419890e-04, 2.7533537e-01, ..., 2.0322010e-02,
        5.3871347e-05, 9.5453332e-05],
       [1.1284639e-02, 9.5170397e-01, 8.07795

Then it's evaluated

In [10]:
evals = model.evaluate([x_test, t_test])

print(evals)

INFO:tensorflow:Restoring parameters from /home/daibak/Documents/Code/Python/aca-tensorflow/cnn/models/dense_cnn/model.ckpt
[{'logits': array([[  81.05848 ,  -62.064182,  -43.652035, ...,  -53.434326,
        -106.73805 , -121.6701  ],
       [ -63.58989 ,  -46.279205,  -69.02652 , ...,   46.97762 ,
        -134.47418 , -105.88981 ],
       [  91.31454 ,  -15.858677,  -87.60347 , ...,  -76.14768 ,
         147.22803 ,   51.774258],
       ...,
       [ -22.75338 ,   -7.063589,  -56.86931 , ...,  107.1757  ,
         -97.29552 ,  -37.601997],
       [  33.548573,  113.97623 , -139.0345  , ...,   60.917297,
         -21.515625,  165.21643 ],
       [  11.03273 ,  -16.96773 ,  -55.12688 , ...,  -70.942825,
          18.96115 ,   64.09692 ]], dtype=float32), 'classes': array([0, 3, 8, ..., 7, 9, 3]), 'probabilities': array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 6.668

## Quantization Transform

In [3]:
graph, inp, out = load_frozen_graph(
    (MODELS_DIR / model.name / 'model.pb').as_posix())

In [3]:
inp, out = (['features'], ['softmax'])

In [4]:
inp, out

(['features'], ['softmax'])

In [ ]:
graph=model.predict_ops_graph[1]

In [5]:
opt_graph = model.optimize_for_inference(add_transf=[])
opt_graph.get_operations()

[<tf.Operation 'logits/bias' type=VariableV2>,
 <tf.Operation 'logits/kernel' type=VariableV2>,
 <tf.Operation 'dense/bias' type=VariableV2>,
 <tf.Operation 'dense/kernel' type=VariableV2>,
 <tf.Operation 'flatten/Reshape/shape/1' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack_2' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack_1' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack' type=Const>,
 <tf.Operation 'conv2d_1/bias' type=VariableV2>,
 <tf.Operation 'conv2d_1/kernel' type=VariableV2>,
 <tf.Operation 'conv2d/bias' type=VariableV2>,
 <tf.Operation 'conv2d/kernel' type=VariableV2>,
 <tf.Operation 'conv2d/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d/MaxPool' type=MaxPool>,
 <tf.Operation 'conv2d_1/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d_1/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d_1/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d_1/MaxPool' type=MaxPool>,


In [13]:
graph.get_operations()

[<tf.Operation 'train_mode' type=Placeholder>,
 <tf.Operation 'features' type=Placeholder>,
 <tf.Operation 'conv2d/kernel' type=Const>,
 <tf.Operation 'conv2d/kernel/read' type=Identity>,
 <tf.Operation 'conv2d/bias' type=Const>,
 <tf.Operation 'conv2d/bias/read' type=Identity>,
 <tf.Operation 'conv2d/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d/MaxPool' type=MaxPool>,
 <tf.Operation 'conv2d_1/kernel' type=Const>,
 <tf.Operation 'conv2d_1/kernel/read' type=Identity>,
 <tf.Operation 'conv2d_1/bias' type=Const>,
 <tf.Operation 'conv2d_1/bias/read' type=Identity>,
 <tf.Operation 'conv2d_1/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d_1/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d_1/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d_1/MaxPool' type=MaxPool>,
 <tf.Operation 'flatten/Shape' type=Shape>,
 <tf.Operation 'flatten/strided_slice/stack' type=Const>,
 <tf.Operation 'flatten/strided_slice/st

### temp

In [10]:
with tf.Session() as sess:
    h = tf.placeholder(tf.bool, shape=(), name='hungry')
    f = tf.placeholder(tf.bool, shape=(), name='fed')
    n = tf.logical_not(f)
    o = sess.run(n, feed_dict={f: False})
o
f.name

'fed_4:0'